# Evolutionary rescue of positive interactions - Models

This notebook contains the implementation of the models describing the dynamics of populations engaged in positive interactions, and code used to simulated the evolutionary rescue senario. 

The notebook is divided into three sections, each describes different type of interaction:
- **Intraspecies cooperation**
- **Interspecies mutualism** 
- **Cooperation in the presence of cheaters**. 

Full description of the models and code can be found in **Section 2** in the supplementary information and the **Methods** section in the manuscript:

https://www.biorxiv.org/content/10.1101/2020.08.06.239608v1


In [1]:
# Imports

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from scipy.integrate import odeint

import seaborn as sns
from cycler import cycler

In [2]:
# Parameters

K = 1e6 # Carrying capacity (#individuals)
N1_initial = (K/2) # Species  1 ancestor initial population density (#individuals)
N2_initial = (K/2) # Species  2 ancestor initial population density (#individuals)
Nc = 1e3 # Critical population size (#individuals)
mu = 1e-6 # Mutation rate (time^-1)

r1 = 0.9 # Growth rate of species 1 ancestor (time^-1) 
r2 = 0.9 # Growth rate of species 2 ancestor (time^-1) 
rm1 = 1.6 # Growth rate of species 1 mutant (time^-1)
rm2 = 1.6 # Growth rate of species 2 mutant (time^-1)
d = 1 # External death rate (time^-1)
ro = 0.62 # Growth rate decrease at low densities (time^-1)

a = 0.01 # Cheaters growth advantage at high cooperator density (time^-1)
b = 0.99 # Cooperators growth advantage at low densities (time^-1)
c = 0 # Cost (time^-1)

d_initial = 0.1 # Cheaters simulation initial death rate (time^-1)
d_stress = 0.6 # Cheaters simulation stress death rate (time^-1)

t = 500 # Simulations time 
dt = 0.01 # Simulations time intervals
stress_onset = 0 # Time point in which stress begins

### Derivative
Phenomenological model of competition under limitation of common resource:

$$\frac{dN_i}{dt} = r_i N_i(1-\frac{\sum{N_j}}{K}) - \delta N_i$$


In [3]:
def derivative(N,t,r,K,d):
    '''
    The derivative used in simulations
        N - Population sizes
        t - time 
        r - Growth rates
        K - Carrying capacity
        d - Death rate 
    '''
    dN = r*N*(1-(np.sum(N))/K)- d*N
    return dN

### Mutate

Poisson point process distribution to sample the number of mutations occurred at a given timepoint:

$$ m \sim Pois(\mu N \Delta t)$$






In [4]:
def mutate(N,mu,dt):
    ''' 
    Calculate expected number of mutations and returns number of mutations occurred 
        N - Population size
        mu - mutation rate
        dt - time interval
    return number of mutations occurred 
    '''
    m = np.round(np.random.poisson(N*mu*dt))
    return m

# Intraspecies cooepration 

### Density dependent growth rate of cooperating species

Implementation of allee effect with density dependent growth rate:

$$\quad r_{l}{\scriptstyle (N_T)}= \begin{cases} r_l & \ \  N_T>N_c \\r_l(1-\rho ) & \ \ N_T<N_c \end{cases} \quad \quad {\scriptsize l \in \{A,M\}}$$



In [5]:
def cooperation_growth_rate(N,ra,rm,ro,Nc):
    ''' 
    Return the density dependent growth rates of cooperating species under Allee effect
        N - Population size
        ra - Ancestor growth rates
        rm - Mutant growth rate
        ro - Growth rate decrease at low densities
        Nc - Critical population size
    returns 2d array of growth rate of ancestors and mutants
    '''
    
    if N[0] + N[1] < Nc:
        r_update = np.array([ra*(1-ro),rm*(1-ro)])
    else:
        r_update = np.array([ra,rm])
        
    return r_update

In [6]:
def plot_simulation_cooperation(N,ax):
    ''' 
    plot simulation of Intraspecies cooperation 
        N - Population size
        ax - Axis for plot
    '''
    
    plt.rc('axes', prop_cycle=(cycler(color=['royalblue','lightblue']) +
                           cycler(linestyle=np.repeat(['-', '--'],[1,1]))))
    
    ax.plot(np.arange(N.shape[0])*dt,N,lw=1.5)
    
    ax.legend(['Ancestor','Mutant'],loc='best')
    ax.set_xlabel('time')
    ax.set_yscale('log')
    ax.set_ylabel('Population density (log)')

In [7]:
def simulate_cooperation(model=derivative,t=t,dt=dt,mu=mu,Nc=Nc,plot_bool=False,r1=r1,rm1=rm1,d=d,c=c,ro=ro,stress_onset=stress_onset,ax=None): 
    ''' 
    Runs simulation of intraspecies cooperation
        model - The derivative
        t - time of simulation
        dt - time intervals
        mu - mutation rate
        Nc - Critical population size
        plot_bool - plot true/false
        r1 - Ancestor growth rate
        rm1 - Mutant growth rate
        ro - Growth rate decrease at low densities
        d - External death rate
        c - Cost
        stress_onset - Time point in which stress begins
        ax - Axis for plot
    return 1 if rescued, 0 if extinct, -1 else
    '''
    
    N = np.array([K,0])
    r = np.array([r1*(1-c),rm1*(1-c)])
    d_cur = 0 
    
    inc = 0 # Does mutant denisty increases
    flag = -1 # 1 if rescued, 0 if extinct, -1 else

    if plot_bool:
        N_plot = np.zeros((int(t/dt)+1,2))
        N_plot[0] = N
        count = 1
    
    # Run simulation in time intervals
    for i in np.linspace(0,t-dt,int(t/dt)):
        if i == stress_onset:
            d_cur = d
            
        # Get derivative
        N = odeint(model,N,[i,i+dt],args=(r,K,d_cur))[-1] 
        N[N<1] = 0
        
        # Calculate growth rate
        r = cooperation_growth_rate(N,r1,rm1,ro,Nc)

        # Check for extinciton
        if ((N[0] < 1 and N[1] < 1) ):
            flag = 0
            if not(plot_bool):
                break

        # Get mutation event
        if(N[1]==0 and N[0]!=0 and d_cur!=0):     
            m1 = np.min([1,mutate(N[0],mu,dt)])
            N[0] = N[0] - m1   
            N[1] = m1
        
        # Check for rescue
        if ((N[1] > Nc)):
            flag = 1
            if not(plot_bool):
                break
                
        # Check for future rescue
        if(i >= t-1 and N[1] >1 and N[0]<Nc):

            if inc == 0 :
                inc = N[1] 

            elif N[1] >= inc:
                flag = 1
                if not(plot_bool):
                    break
            else:
                flag = 0
                if not(plot_bool):
                    break
                    
        if(plot_bool):
            N_plot[count] = N
            count = count+1

            
    if(i==t-dt and not(plot_bool)):
        print('No result t = ' + str(i))
        flag = -1
        
    if(plot_bool):
        plot_simulation_cooperation(N_plot,ax)
        return flag, N_plot
    
    return flag

# Mutualism

### Density dependent growth rate of mutualistic species

Implementation  of allee effect with density dependent growth rate:


$$ \quad r_{li}{\scriptstyle (N_{Tj})}= \begin{cases} r_{li} & \ \  N_{Tj}>N_c \\r_{li}(1-\rho ) & \ \ N_{Tj}<N_c \end{cases} \quad \quad {\scriptsize l \in \{A,M\}},\ {\scriptsize i,j \in \{1,2\}}$$

In [8]:
def mutualism_growth_rate(N,r1,r2,rm1,rm2,ro,Nc):
    ''' 
    Return density dependent growth rates of mutualistic species under Allee effect
        N - Population size
        r1 - Species 1 ancestor growth rate
        r2 - Species 2 ancestor growth rate
        rm1 - Species 1 mutant growth rate
        rm2 - Species 2 mutant growth rate
        ro - Growth rate decrease at low densities
        Nc - critical population size
    returns 4d array of growth rate of ancestors and mutants [r1,r2,rm1,rm2]
    '''
    
    r_updated = np.zeros(4)
    if N[0] + N[2] < Nc:
        r_updated[1] = r2*(1-ro)
        r_updated[3] = rm2*(1-ro)
    else:
        r_updated[1] = r2
        r_updated[3] = rm2
    if N[1] + N[3] < Nc: 
        r_updated[0] = r1*(1-ro)
        r_updated[2] = rm1*(1-ro)
    else:
        r_updated[0] = r1
        r_updated[2] = rm1
    return r_updated

In [9]:
def plot_simulation_mutualism(N,ax):
    ''' 
    plot simulation of mutualism
        N - Population size
        ax - Axis for plot
    '''

    plt.rc('axes', prop_cycle=(cycler(color=['steelblue', 'coral','steelblue', 'coral']) +
                           cycler(linestyle=np.repeat(['-', '--'],[2,2]))))
    ax.plot(np.arange(N.shape[0])*dt,N[:,:],lw=1.5)
    

    ax.set_xlabel('time')
    ax.set_ylabel('Population density (log)')
    ax.set_yticks([])
    ax.legend(['Ancestor 1','Ancestor 2','Mutant 1','Mutant 2'],loc='best')
    ax.grid(False)
    ax.set_yscale('log')
    

In [10]:
def simulate_mutualism(model=derivative,t=t,dt=dt,mu=mu,Nc=Nc,plot_bool=False,r1=r1,r2=r2,rm1=rm1,rm2=rm2,ro=ro,d=d,c=c,stress_onset=stress_onset,ax=None): 
    ''' 
    Runs simulation of mutualism
        model - The derivative
        t - time of simulation
        dt - time intervals
        mu - mutation rate
        Nc - Critical population size
        plot_bool - plot true/false
        r1 - Species 1 ancestor growth rate
        r2 - Species 2 ancestor growth rate
        rm1 - Species 1 mutant growth rate
        rm2 - Species 2 mutant growth rate
        ro - Growth rate decrease at low densities
        d - External death rate
        c - Cost
        stress_onset - Time point in which stress begins
        ax - Axis for plot
    return 1 if rescued, 0 if extinct, -1 else
    '''
    
    N = np.array([N1_initial,N2_initial,0,0])
    r = np.array([r1*(1-c),r2*(1-c),rm1*(1-c),rm2*(1-c)])
    flag = -1 # 1 if rescued, 0 if extinct, -1 else
    d_cur=0

    if plot_bool:
        N_plot = np.zeros((int(t/dt)+1,4))
        N_plot[0] = N
        count = 1
    
    # Run simulation in time intervals
    for i in np.linspace(0,t-dt,int(t/dt)):
        if i == stress_onset :
            
            d_cur = d
        N = odeint(model,N,[i,i+dt],args=(r,K,d_cur))[-1]        
        N[N<1] = 0
        
        # Calculate growth rate
        r = mutualism_growth_rate(N,r1,r2,rm1,rm2,ro,Nc)
        
        # Check for extinction
        if ((N[0] <1 and N[2]<1) or (N[1]<1 and N[3] < 1)):
            flag = 0
            if not(plot_bool):
                break
        # Get mutation events
        if(N[2]==0 and N[0]!=0 and d_cur!=0):
            m1 = np.min([1,mutate(N[0],mu,dt)])
            N[0] = N[0] - m1
            N[2] = m1
        if(N[3]==0 and N[1]!=0 and d_cur!=0):    
            m2 = np.min([1,mutate(N[1],mu,dt)])                 
            N[1] = N[1] - m2
            N[3] = m2
            
        # Check for rescue
        if (N[2] > Nc) and (N[3] > Nc):
            flag = 1
            if not(plot_bool):
                break
                
        if(plot_bool):
            N_plot[count] = N
            count = count+1

    if(i==t-dt and not(plot_bool)):
        print('No result t = ' + str(i))
        flag = -1
    if(plot_bool):
        plot_simulation_mutualism(N_plot,ax)
        return flag, N_plot
    return flag
    




# Cheaters

### Density dependent growth rate of cooperating species in the presence of cheaters

Implementation  of allee effect with density dependent growth rate:


$$ \quad r_{coop,l}{\scriptstyle (N_{coop})}= \begin{cases} r_{coop,l} & \ \  N_{coop}>N_c \\r_{coop,l}(1-\rho ) & \ \ N_{coop}<N_c \end{cases} \quad \quad {\scriptsize l \in \{A,M\}}$$
$$ \quad r_{cheat,l}{\scriptstyle (N_{coop})}= \begin{cases} r_{cheat,l}(1+b) & \ \  N_{coop}>N_c \\r_{cheat,l}(1-\rho )(1-a) & \ \ N_{coop}<N_c \end{cases} \quad \quad {\scriptsize l \in \{A,M\}}$$

In [11]:
def cheaters_growth_rates(N,r1,r2,rm1,rm2,ro1,ro2,a,b):
    ''' 
    Return density dependent growth rates of cooperating species and cheaters
        N - Population size
        r1 - Cooperator ancestor growth rates
        rm1 - Cooperator mutant growth rates
        r2 - Cheater ancestor growth rates
        rm2 - Cheater mutant growth rates        
        ro - Growth rate decrease at low densities
        a - Cheaters growth advantage at high cooperator density
        b - Cooperators growth advantage at low densities 
        Nc - critical population size
    returns 4d array of growth rate of ancestors and mutants [r1,r2,rm1,rm2]
    '''
    r_updated = np.zeros(4)
    
    if N[0] + N[2] < Nc:
            r_updated[1] = r2*(1-ro2)*(1-a)
            r_updated[3] = rm2*(1-ro2)*(1-a)
            r_updated[0] = r1*(1-ro1)
            r_updated[2] = rm1*(1-ro1)
    else:
            r_updated[1] = r2*(1+b)
            r_updated[3] = rm2*(1+b)
            r_updated[0] = r1
            r_updated[2] = rm1
    return r_updated

In [12]:
def plot_simulation_cheaters(N_plot,ax):
    ''' 
    plot simulation with cheaters
        N - Population size
        ax - Axis for plot
    '''
    plt.rc('axes', prop_cycle=(cycler(color=['steelblue', 'coral','steelblue', 'coral']) +
                           cycler(linestyle=np.repeat(['-', '--'],[2,2]))))
    ax.plot(np.arange(N_plot.shape[0])*dt,N_plot[:,:],lw=lines_big)
    

    ax.set_xlabel('time')
    ax.set_ylabel('Population size')
    ax.set_yticks([])
    ax.set_yscale('log')
    ax.legend(['Coop ancestor','Cheat ancestor','Coop mutant','Cheat mutant'],loc='best')
    
    

In [13]:
def simulate_cheaters(model=derivative,t=t,dt=dt,mu=mu,Nc=Nc,N1_initial=N1_initial,N2_initial=N2_initial,plot_bool=False,r1=r1,r2=r2,rm1=rm1,rm2=rm2,ro1=ro,ro2=ro,d_initial=d_initial,d_stress=d_stress,a=a,b=b,stress_onset=stress_onset,ax=None): 
    ''' 
    Runs simulation of mutualism
        model - The derivative
        t - time of simulation
        dt - time intervals
        mu - mutation rate
        Nc - Critical population size
        plot_bool - plot true/false
        r1 - Species 1 ancestor growth rate
        r2 - Species 2 ancestor growth rate
        rm1 - Species 1 mutant growth rate
        rm2 - Species 2 mutant growth rate
        ro1 - Species 1 growth rate decrease at low densities
        ro2 - Species 2 growth rate decrease at low densities
        d - External death rate
        c - Cost
        a - Cheaters growth advantage at high cooperator density
        b - Cooperators growth advantage at low densities 
        d_initial - Initial death rate 
        d_stress -  Death rate after stress onset
        stress_onset - Time point in which stress begins
        ax - Axis for plot
    return 1 if rescued, 0 if extinct, -1 else
    '''
    rm1 = rm1*(1-c)
    r1=r1*(1-c)
    rm2 = rm2*(1-c)
    r2=r2*(1-c)
    d = d_initial
    N = np.array([N1_initial,N2_initial,0,0])
    r = np.array([r1,r2,rm1,rm2])
    flag = -1  #1 if rescued, 0 if extinct, -1 else
    

    if plot_bool:
        N_plot = np.zeros((int(t/dt)+1,4))
        N_plot[0] = N
        count = 1
    
    # Run simulation in time intervals
    for i in np.linspace(0,t-dt,int(t/dt)):
        if i == stress_onset :
            d = d_stress
        N = sol = odeint(model,N,[i,i+dt],args=(r,K,d))[-1]       
        N[N<1] = 0

        # Calculate growth rate
        r = cheaters_growth_rates(N,r1,r2,rm1,rm2,ro1,ro2,a,b)
        
        # Check for extinction
        if ((N[0] <1 and N[2]<1)):
            flag = 0
            if not(plot_bool):
                break        
            
        # Get mutation events            
        if(N[2]==0 and N[0]!=0 and d!=d_initial):
            m1 = np.min([1,mutate(N[0],mu,dt)])
            N[0] = N[0] - m1
            N[2] = m1
            
        if(N[3]==0 and N[1]!=0 and d!=d_initial):          
            m2 = np.min([1,mutate(N[1],mu,dt)])
            N[1] = N[1] - m2
            N[3] = m2
            
        # Check for rescue
        if (N[2] > Nc):
            if ((N[1] <1 and N[3]<1)):
                flag = 1
            else:
                flag =0
                
            if not(plot_bool):
                    break
                    
        # Check for extinction
        if (N[0]<1 and N[2] < 1):        
            flag = 0
            if not(plot_bool):
                break
                
        if(plot_bool):
            N_plot[count] = N
            count = count+1

    if(i==t-dt and not(plot_bool)):
        print('No result t = ' + str(i))
        if flag !=2:
            flag = -1
    if(plot_bool):
        
        plot_simulation_cheaters(N_plot,ax)
        return flag, N_plot
    return flag
    


